In [1]:
import torch

In [6]:
torch.randn(1)[0]

tensor(0.2088)

In [8]:
torch.tensor(1.0).size()

torch.Size([])

In [19]:
sample = torch.randn(2)
timesteps = torch.randn([])
if not torch.is_tensor(timesteps):
    # TODO: this requires sync between CPU and GPU. So try to pass timesteps as tensors if you can
    # This would be a good case for the `match` statement (Python 3.10+)
    is_mps = sample.device.type == "mps"
    if isinstance(timesteps, float):
        dtype = torch.float32 if is_mps else torch.float64
    else:
        dtype = torch.int32 if is_mps else torch.int64
    timesteps = torch.tensor([timesteps], dtype=dtype, device=sample.device)
elif len(timesteps.shape) == 0:
    timesteps = timesteps[None].to(sample.device)
    print("run here")

# broadcast to batch dimension in a way that's compatible with ONNX/Core ML
timesteps = timesteps.expand(sample.shape[0]) # ensure that the batch dimension is the same as the sample dimension
timesteps

run here


tensor([0.0927, 0.0927])

In [16]:
timesteps.shape

torch.Size([2])

In [2]:
from torch.onnx import export
from diffusers import ControlNetModel
from pathlib import Path
import torch

model_path = "diffusers/controlnet-canny-sdxl-1.0"
attention_slicing = None

controlnet = ControlNetModel.from_pretrained(model_path, addition_embed_type=None)
if attention_slicing is not None:
    print("Attention slicing: " + attention_slicing)
    controlnet.set_attention_slice(attention_slicing)

dtype=torch.float32
device = "cpu"

output_path = Path("./dumps")
cnet_path = output_path / "controlnet" / "model.onnx"

model_args = (
    torch.randn(2, 4, 64, 64).to(device=device, dtype=dtype),
    torch.randn(1).to(device=device, dtype=dtype),
    torch.randn(2, 77, 768).to(device=device, dtype=dtype),
    torch.randn(2, 3, 512,512).to(device=device, dtype=dtype),
    torch.randn(1).to(device=device, dtype=dtype), # controlnet_cond
    None, None, None, None, None, # class_labels, timestep_cond, attention_mask, added_cond_kwargs, cross_attention_kwargs
    False, False # guess_mode, return_dict
)
ordered_input_names =["sample", "timestep", "encoder_hidden_states", "controlnet_cond"]
print("Exporting controlnet to ONNX...")
export(
    controlnet,
    model_args,
    f=output_path.as_posix(),
    input_names=ordered_input_names,
    dynamic_axes={
        "sample": {0: "batch", 1: "channels", 2: "height", 3: "width"},
        "encoder_hidden_states": {0: "batch", 1: "sequence"},
        "controlnet_cond": {0: "batch", 2: "height", 3: "width"},
    }, #* timestep and condition_scale are not dynamic axes
    opset_version=14,
    export_params=True,
)

Some weights of the model checkpoint were not used when initializing ControlNetModel: 
 ['add_embedding.linear_1.bias, add_embedding.linear_1.weight, add_embedding.linear_2.bias, add_embedding.linear_2.weight']


Exporting controlnet to ONNX...


/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels


RuntimeError: mat1 and mat2 shapes cannot be multiplied (154x768 and 2048x640)

In [8]:
from sdserve.convert.controlnet.pt2onnx import convert_controlnet



convert_controlnet(
    ,
    output_path / "controlnet-canny-sdxl-1.0.onnx",
    opset=14,
    fp16=False,
    attention_slicing="max",
)

Attention slicing: max
Exporting controlnet to ONNX...


TypeError: argument of type 'NoneType' is not iterable

In [5]:
from pathlib import Path
import transformers
import torch
from transformers.onnx import FeaturesManager
from diffusers import ControlNetModel

# load model and tokenizer
feature = "text-to-Iimage"
model_path = "diffusers/controlnet-canny-sdxl-1.0"
controlnet = ControlNetModel.from_pretrained(model_path)
attention_slicing = None

controlnet = ControlNetModel.from_pretrained(model_path, addition_embed_type=None)
if attention_slicing is not None:
    print("Attention slicing: " + attention_slicing)
    controlnet.set_attention_slice(attention_slicing)

dtype=torch.float32
device = "cpu"

output_path = Path("./dumps")
cnet_path = output_path / "controlnet" / "model.onnx"
# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(controlnet)
onnx_config = model_onnx_config(controlnet.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
    model=controlnet,
    config=onnx_config,
    opset=13,
    output=cnet_path
)

Some weights of the model checkpoint were not used when initializing ControlNetModel: 
 ['add_embedding.linear_1.bias, add_embedding.linear_1.weight, add_embedding.linear_2.bias, add_embedding.linear_2.weight']


AttributeError: 'FrozenDict' object has no attribute 'model_type'

In [3]:
from optimum.onnxruntime import ORTStableDiffusionXLPipeline

model_path = "diffusers/controlnet-canny-sdxl-1.0"
model = ORTStableDiffusionXLPipeline.from_pretrained(model_path, from_transformers=True)


The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
model_index.json not found in the specified subfolder . Using the top level model_index.json.


OSError: diffusers/controlnet-canny-sdxl-1.0 does not appear to have a file named model_index.json.

In [ ]:
import argparse
import os
import shutil
from pathlib import Path

import onnx
import onnx_graphsurgeon as gs
import torch
from onnx import shape_inference
from packaging import version
from polygraphy.backend.onnx.loader import fold_constants
from torch.onnx import export

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetImg2ImgPipeline,
)
from diffusers.models.attention_processor import AttnProcessor
from diffusers.pipelines.controlnet.pipeline_controlnet_sd_xl import StableDiffusionXLControlNetPipeline


is_torch_less_than_1_11 = version.parse(version.parse(torch.__version__).base_version) < version.parse("1.11")
is_torch_2_0_1 = version.parse(version.parse(torch.__version__).base_version) == version.parse("2.0.1")

model_path='stabilityai/stable-diffusion-xl-base-1.0'
controlnet_path = ["diffusers/controlnet-canny-sdxl-1.0"]

controlnets = []
for path in controlnet_path:
    controlnet = ControlNetModel.from_pretrained(path, torch_dtype=dtype).to(device)
    if is_torch_2_0_1:
        controlnet.set_attn_processor(AttnProcessor())
    controlnets.append(controlnet)


if len(controlnets) == 1:
    controlnet = controlnets[0]
else:
    raise ValueError("MultiControlNet is not yet supported.")
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
    model_path, controlnet=controlnet, torch_dtype=dtype, variant="fp16", use_safetensors=True
).to(device)

In [8]:
!optimum-cli export onnx --model diffusers/controlnet-canny-sdxl-1.0 ./dumps/  --framework pt

Traceback (most recent call last):
  File "/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main/model_index.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/configuration_utils.py", line 379, in load_config
    config_file = hf_hub_download(
  File "/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py", line 101, in inner_f
    return f(*args, **kwargs)
  File "/home/tie

In [1]:
import argparse
import os
import shutil
from pathlib import Path

import onnx
import onnx_graphsurgeon as gs
import torch
from onnx import shape_inference
from packaging import version
from polygraphy.backend.onnx.loader import fold_constants
from torch.onnx import export

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetImg2ImgPipeline,
)
from diffusers.models.attention_processor import AttnProcessor
from diffusers.pipelines.controlnet.pipeline_controlnet_sd_xl import StableDiffusionXLControlNetPipeline

dtype=torch.float16
device="cuda"

is_torch_less_than_1_11 = version.parse(version.parse(torch.__version__).base_version) < version.parse("1.11")
is_torch_2_0_1 = version.parse(version.parse(torch.__version__).base_version) == version.parse("2.0.1")

model_path='stabilityai/stable-diffusion-xl-base-1.0'
controlnet_path = ["diffusers/controlnet-canny-sdxl-1.0"]

controlnets = []
for path in controlnet_path:
    controlnet = ControlNetModel.from_pretrained(path, torch_dtype=dtype).to(device)
    if is_torch_2_0_1:
        controlnet.set_attn_processor(AttnProcessor())
    controlnets.append(controlnet)


if len(controlnets) == 1:
    controlnet = controlnets[0]
else:
    raise ValueError("MultiControlNet is not yet supported.")
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
    model_path, controlnet=controlnet, torch_dtype=dtype, variant="fp16", use_safetensors=True
).to(device)

/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.02it/s]


In [2]:
controlnet = pipeline.controlnet
del pipeline
torch.cuda.empty_cache()

In [3]:
control_model_input = torch.randn(2, 4, 128, 128, dtype=torch.float16, device="cuda")
controlnet_prompt_embeds = torch.randn(2, 77, 2048, dtype=torch.float16, device="cuda")
controlnet_added_cond_kwargs = {
    'text_embeds': torch.randn(2, 1280, dtype=torch.float16, device="cuda"), 
    'time_ids': torch.randn(2, 6, dtype=torch.float16, device="cuda")
}
t=torch.tensor(841., device='cuda:0')
cond_scale=1.0
image = torch.randn(2, 3, 1024, 1024, dtype=torch.float16, device="cuda")

In [ ]:
import torch

torch.onnx.export(
    controlnet,
    (control_model_input, t, controlnet_prompt_embeds, image, cond_scale, None, None, None, controlnet_added_cond_kwargs, None, False, False),
    "dumps/controlnetsdxl.onnx",
    input_names=[
        "control_model_input", 
        "t", 
        "controlnet_prompt_embeds", 
        "controlnet_image", 
        "cond_scale", 
        "",
        "",
        "",
        "added_cond_kwargs", 
        "",
        "",
        "",
      ],
    output_names=["down_block", "mid_block"],
    dynamic_axes={
        "control_model_input": {0: "batch", 2: "height", 3: "width"},
        "controlnet_prompt_embeds": {0: "batch", 1: "sequence"},
        "controlnet_image": {0: "batch", 2: "height", 3: "width"},
        "down_block": {0: "batch", 1: "sequence", 2: "height", 3: "width"},
        "mid_block": {0: "batch", 1: "sequence", 2: "height", 3: "width"},
    },
    export_params=False,
    opset_version=14,
    verbose=True,
)

/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/controlnet.py:844: TracerWarning: Converting a tensor to a Python boolean might c

In [ ]:
import onnx

onnx_model = onnx.load("dumps/controlnetsdxl.onnx")

In [ ]:

# Function to get tensor information
def print_tensor_info(tensor):
    name = tensor.name
    shape = tensor.type.tensor_type.shape
    data_type = tensor.type.tensor_type.elem_type
    shape = [dim.dim_value for dim in shape.dim]
    
    # Convert ONNX data type to human-readable format
    dtype_map = {
        1: 'float32',
        2: 'uint8',
        3: 'int8',
        4: 'uint16',
        5: 'int16',
        6: 'int32',
        7: 'int64',
        8: 'uint32',
        9: 'uint64',
        10: 'float64',
        11: 'string',
        12: 'bool',
        # Add other types if necessary
    }
    
    dtype = dtype_map.get(data_type, 'unknown')
    
    print(f"Name: {name}, Shape: {shape}, Data Type: {dtype}")

# Print information for all inputs
print("Inputs:")
for input in model.graph.input:
    print_tensor_info(input)

# Print information for all outputs
print("\nOutputs:")
for output in model.graph.output:
    print_tensor_info(output)
